## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
    
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)

if response is not None:
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table', {'class':'wikitable sortable'}).tbody
    
rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

wiki_postcodes = pd.DataFrame(columns=columns)

for i in range (1, len(rows)):
    tds = rows[i].find_all('td')
    
    values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
    
    wiki_postcodes = wiki_postcodes.append(pd.Series(values, index=columns), ignore_index=True)


wiki_postcodes.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [2]:
wiki_postcodes.rename(columns = {'Postcode':'PostalCode'}, inplace = True)

wiki_postcodes.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
indexNames = wiki_postcodes[wiki_postcodes['Borough'] == 'Not assigned' ].index
wiki_postcodes.drop(indexNames , inplace=True)

# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
wiki_postcodes = wiki_postcodes.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
wiki_postcodes['Neighbourhood'].loc[wiki_postcodes['Neighbourhood'] == 'Not assigned'] = wiki_postcodes['Borough']

wiki_postcodes.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
wiki_postcodes.shape

(103, 3)